In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

servico = Service(ChromeDriverManager().install())

import time
import pandas as pd
from tqdm import tqdm

## Livelo

In [ ]:
def promocoes_livelo(lista_sites):
    
    ''''
    Realizar a procura dos pontos, sites, links e regulamentos dos parceiros na Livelo
    
    Parâmetros:
    lista_sites : a lista dos parceiros para serem procurados
    
    '''
    
    promocoes_livelo = pd.DataFrame(columns=[['Site','Pontos','Regulamento', 'Link']])
    
    
    #criar a barra de progresso
    pbar = tqdm(lista_sites)
    
    for site in pbar:
                
        #Entrar no site da Livelo
        
        while True:
            try:
                navegador.get('https://www.livelo.com.br/')
                break
            except:
                continue
        
        pbar.set_description(f'Pesquisando {site}...')
        
        #Esperar o PopUp aparecer e fechar ele
        while len(navegador.find_elements(By.XPATH, '//*[@id="div-footer"]/section/div/div[3]/span')) ==0:
            time.sleep(1)
        navegador.find_element(By.XPATH, '//*[@id="div-footer"]/section/div/div[3]/span').click()

        #Clicar nas opcções até entrar na listagem dos sites pareceiros
        botao = navegador.find_element(By.ID, 'li-first-level-menu-2')
        ActionChains(navegador).move_to_element(botao).perform()
        time.sleep(0.3)
        botao2 = navegador.find_element(By.ID, 'li-second-level-menu-2-1')
        time.sleep(0.3)
        ActionChains(navegador).move_to_element(botao2).perform()
        botao3 = navegador.find_element(By.ID, 'li-third-level-menu-2-1-5')
        ActionChains(navegador).move_to_element(botao3).perform()
        time.sleep(0.5)
        navegador.find_element(By.ID, 'li-third-level-menu-2-1-5').click()
        while len(navegador.find_elements(By.ID, 'inpt-filterSearch')) ==0:
            time.sleep(1)
        
        #Procurar o parceiro desejado
        navegador.find_element(By.ID, 'inpt-filterSearch').send_keys(site, Keys.ENTER)
        navegador.find_element(By.XPATH, '//*[@id="btn-partnerSearch"]').click()
        try:
            navegador.find_element(By.CLASS_NAME, 'button__knowmore--info').click()
        except:
            continue
        
        while len(navegador.find_elements(By.XPATH, '//*[@id="div-bannerParityInfo"]/div[1]/span[4]')) == 0:
            time.sleep(1)
        
        #Pegar os pontos
        try:
            pontos = navegador.find_element(By.CLASS_NAME, 'parity__card--info_club-span-points').text
        except:
            pontos = navegador.find_element(By.XPATH, '//*[@id="div-bannerParityInfo"]/div[1]/span[4]').text
        
        #Pegar o regulamento
        try:
            regulamento = navegador.find_element(By.CLASS_NAME, 'banner__blocktext-text').text
        except:
            regulamento = navegador.find_element(By.ID, 'h1-bannerParityMainText').text
         
        #Pegar o link
        try:
            link = navegador.find_element(By.CLASS_NAME, 'icon-external-link').get_attribute('href')
        except:
            continue
            
        #Criar a tabela auxiliar
        tabela_intermediaria = pd.DataFrame([[site, pontos, regulamento, link]], columns=[['Site','Pontos','Regulamento','Link']])
        pd.set_option('display.max_colwidth', None)
        
        #Adicionar na tabela promoções
        promocoes_livelo = promocoes_livelo.append(tabela_intermediaria)
        pd.set_option('display.max_colwidth', None)
      
        
        
    return promocoes_livelo

In [ ]:
#Entrar com os parceiros desejados
lista_sites = []
site =input('Digite o site desejado (0 para encerrar): ')



while site != '0':
    lista_sites.append(site.capitalize())
    site =input('Digite o site desejado (0 para encerrar): ')

time.sleep(1)
print('\n')
print('Situações da Livelo:\n')

#criar o navegador
navegador = webdriver.Chrome(service=servico)
navegador.maximize_window()

#Procurar os parceiros
display(promocoes_livelo(lista_sites))

#Fechar o navegador
navegador.quit()